In [ ]:
%cd /home/ubuntu/Qwen-Image-Edit-Angles

In [ ]:
import torch
from qwenimage.debug import clear_cuda_memory, print_gpu_memory
from qwenimage.experiment import ExperimentConfig
from qwenimage.experiments.experiments_qwen import PipeInputs, Qwen_AoT, QwenBaseExperiment, ExperimentRegistry

from qwenimage.debug import ctimed, ftimed, print_first_param
from qwenimage.models.attention_processors import QwenDoubleStreamAttnProcessorFA3, QwenDoubleStreamAttnProcessorSageAttn2, sageattn_qk_int8_pv_fp16_cuda_wrapper, sageattn_qk_int8_pv_fp16_triton_wrapper, sageattn_qk_int8_pv_fp8_cuda_sm90_wrapper, sageattn_qk_int8_pv_fp8_cuda_wrapper, sageattn_wrapper
from qwenimage.models.first_block_cache import apply_cache_on_pipe
from qwenimage.models.pipeline_qwenimage_edit_plus import QwenImageEditPlusPipeline, calculate_dimensions
from qwenimage.models.transformer_qwenimage import QwenImageTransformer2DModel
from qwenimage.experiment import AbstractExperiment, ExperimentConfig
from qwenimage.debug import ProfileSession, ftimed
from qwenimage.optimization import INDUCTOR_CONFIGS, TRANSFORMER_DYNAMIC_SHAPES, aoti_apply, drain_module_parameters, optimize_pipeline_
from qwenimage.prompt import build_camera_prompt


In [ ]:
name = "qwen_base"

experiment = ExperimentRegistry.get(name)(
    config=ExperimentConfig(
        name=name,
    ), 
)
experiment.load()


In [ ]:
# experiment.pipe.transformer

In [ ]:
experiment.pipe.transformer.fuse_qkv_projections()

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.fuse_projections()

In [ ]:
print(experiment.pipe.transformer.transformer_blocks[0].attn.fused_projections)
experiment.pipe.transformer.transformer_blocks[0].attn

In [ ]:
print(experiment.pipe.transformer.transformer_blocks[1].attn.fused_projections)
experiment.pipe.transformer.transformer_blocks[1].attn

In [ ]:
print([b.attn.fused_projections for b in experiment.pipe.transformer.transformer_blocks])

In [ ]:
for module in experiment.pipe.transformer.modules():
    print(module.__class__)
    # if isinstance(module, AttentionModuleMixin):
    #     module.fuse_projections()

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.processor

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.processor

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.processor.fuse_projections()

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.fuse_projections()

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.to_qkv.weight

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.to_q.weight

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.add_q_proj.weight

In [ ]:
experiment.pipe.transformer.transformer_blocks[0].attn.to_added_qkv.weight

In [ ]:
experiment.pipe.scheduler

In [ ]:

import math
from diffusers.schedulers.scheduling_flow_match_euler_discrete import FlowMatchEulerDiscreteScheduler
import torch

from qwenimage.models.pipeline_qwenimage_edit_plus import QwenImageEditPlusPipeline
from qwenimage.models.transformer_qwenimage import QwenImageTransformer2DModel

# Scheduler configuration for Lightning
scheduler_config = {
    "base_image_seq_len": 256,
    "base_shift": math.log(3),  # We use shift=3 in distillation
    "invert_sigmas": False,
    "max_image_seq_len": 8192,
    "max_shift": math.log(3),  # We use shift=3 in distillation
    "num_train_timesteps": 1000,
    "shift": 1.0,
    "shift_terminal": None,  # set shift_terminal to None
    "stochastic_sampling": False,
    "time_shift_type": "exponential",
    "use_beta_sigmas": False,
    "use_dynamic_shifting": True,
    "use_exponential_sigmas": False,
    "use_karras_sigmas": False,
}

# Initialize scheduler with Lightning config

scheduler = FlowMatchEulerDiscreteScheduler.from_config(scheduler_config) # TODO: check scheduler sync issue mentioned by https://pytorch.org/blog/presenting-flux-fast-making-flux-go-brrr-on-h100s/

dtype = torch.bfloat16
device = "cuda" if torch.cuda.is_available() else "cpu"


pipe = QwenImageEditPlusPipeline.from_pretrained(
    "Qwen/Qwen-Image-Edit-2509", 
    transformer=QwenImageTransformer2DModel.from_pretrained( # TODO: remove this if using lightning
        "linoyts/Qwen-Image-Edit-Rapid-AIO", 
        subfolder='transformer',
        torch_dtype=dtype,
        device_map='cuda'),
    scheduler=scheduler,
    torch_dtype=dtype,
).to(device)

In [ ]:
import torch

e = torch.rand((1, 253, 3584), dtype=torch.bfloat16)
e.numel() * e.element_size() / (1024 ** 2)

In [ ]:
dtype = torch.bfloat16
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"experiment load cuda: {torch.cuda.is_available()=}")

pipe = QwenImageEditPlusPipeline.from_pretrained(
    "Qwen/Qwen-Image-Edit-2509", 
    transformer=QwenImageTransformer2DModel.from_pretrained( # use our own model
        "Qwen/Qwen-Image-Edit-2509",
        subfolder='transformer',
        torch_dtype=dtype,
        device_map=device
    ),
    torch_dtype=dtype,
).to(device)

In [ ]:
pipe.scheduler

In [ ]:
from qwenimage.models.pipeline_qwenimage_edit_plus import retrieve_timesteps, calculate_shift
import numpy as np

num_inference_steps = 10

sigmas = np.linspace(1.0, 1 / num_inference_steps, num_inference_steps)
# image_seq_len = latents.shape[1]
image_seq_len = 4096
mu = calculate_shift(
    image_seq_len,
    pipe.scheduler.config.get("base_image_seq_len", 256),
    pipe.scheduler.config.get("max_image_seq_len", 4096),
    pipe.scheduler.config.get("base_shift", 0.5),
    pipe.scheduler.config.get("max_shift", 1.15),
)
timesteps, num_inference_steps = retrieve_timesteps(
    pipe.scheduler,
    num_inference_steps,
    device,
    sigmas=sigmas,
    mu=mu,
)

In [ ]:
timesteps

In [ ]:
from matplotlib import pyplot as plt

plt.plot(timesteps.float().cpu().numpy())

In [ ]:
num_inference_steps

In [ ]:
from qwenimage.models.pipeline_qwenimage_edit_plus import retrieve_timesteps


retrieve_timesteps(pipe.scheduler, num_inference_steps=num_inference_steps)

In [ ]:
pipe.to("cpu")

In [ ]:
from qwenimage.debug import clear_cuda_memory

In [ ]:
clear_cuda_memory()

In [ ]:
pipe.transformer.to("cuda")

In [ ]:
inps = PipeInputs()

In [ ]:
pipe(**inps[0])

In [ ]:
pipe.transformer.__class__.__module__

In [ ]:
pipe.transformer.__class__.__name__

In [ ]:
# pipe.transformer.__class__ = QwenImageTransformer2DModel


In [ ]:
pipe._internal_dict

In [ ]:
pipe._internal_dict["transformer"]

In [1]:
%cd /home/ubuntu/Qwen-Image-Edit-Angles

/home/ubuntu/Qwen-Image-Edit-Angles


In [3]:
from qwenimage.foundation import QwenImageFoundation
from qwenimage.datamodels import QwenInputs, QwenConfig
from qwenimage.experiments.experiments_qwen import PipeInputs
from qwenimage.models.pipeline_qwenimage_edit_plus import QwenImageEditPlusPipeline, calculate_dimensions

In [4]:


foundation = QwenImageFoundation(QwenConfig())

self.device='cuda'


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
from wandml.finetune.lora.lora import LoraFinetuner

foundation.transformer.fuse_qkv_projections()

LoraFinetuner.find_target_modules(foundation.transformer)

override


['to_k',
 'proj_out',
 'linear_2',
 'add_q_proj',
 'net.2',
 'to_added_qkv',
 'linear',
 'to_q',
 'to_v',
 'add_v_proj',
 'img_in',
 'to_add_out',
 'linear_1',
 'txt_in',
 'to_qkv',
 'proj',
 'img_mod.1',
 'txt_mod.1',
 'add_k_proj',
 'to_out.0']

In [4]:


inps = PipeInputs()
im = inps[0]["image"][0]

im = im.resize(calculate_dimensions(1024**2, im.width / im.height))

720 input combinations


In [13]:
im.size

(1248, 832)

In [5]:
# foundation.pipe.vae.to(foundation.dtype)

In [ ]:
latents = foundation.pil_to_latents(im)


In [12]:
latents.shape

torch.Size([1, 16, 1, 104, 156])

In [7]:
images = foundation.latents_to_pil(latents)

In [8]:
type(im), type(images[0])

(PIL.Image.Image, PIL.Image.Image)

In [9]:
import torch.nn.functional as F
import torchvision.transforms.v2 as T
import torch

_transforms = T.Compose([
    T.ToImage(),
    T.RGB(),
    T.ToDtype(torch.float32, scale=True), # [0,1]
    # T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # [-1,1]
])

F.mse_loss(_transforms(im), _transforms(images[0]))

tensor(2.1797e-05)

In [10]:
foundation.dtype

torch.bfloat16

In [11]:
foundation.base_pipe(QwenInputs(**inps[0]))

Time taken by Preprocessing: 0.04293842299375683 seconds


Time taken by QwenImageEditPlusPipeline.encode_prompt: 0.427993256947957 seconds
Time taken by Encode Prompt: 0.42811784299556166 seconds
Time taken by QwenImageEditPlusPipeline._encode_vae_image: 0.07400305196642876 seconds
image_seq_len=4056
mu=0.6915322580645161
timesteps=tensor([1000.0000,  766.4941,  455.3210,   20.0000], device='cuda:0')
Time taken by Prep gen: 0.10688352002762258 seconds


  0%|          | 0/4 [00:00<?, ?it/s]

Time taken by loop 0: 0.49157930293586105 seconds
Time taken by loop 1: 0.37133564590476453 seconds
Time taken by loop 2: 0.44644036900717765 seconds
Time taken by loop 3: 0.44158994196914136 seconds
Time taken by loop: 1.764969157986343 seconds
Time taken by pre decode: 0.07267801999114454 seconds
Time taken by vae.decode: 0.07378141907975078 seconds
Time taken by post process: 0.10780154494568706 seconds
Time taken by offload: 0.0050185970030725 seconds
Time taken by QwenImageEditPlusPipeline.__call__: 2.603054799954407 seconds


[<PIL.Image.Image image mode=RGB size=1248x832>]

In [ ]:
import torch 

im_px = foundation.pipe.image_processor.preprocess(im).unsqueeze(2).to(dtype=torch.bfloat16)
im_px.shape

In [ ]:
foundation.pipe.vae.to(torch.bfloat16)

In [ ]:
latents = foundation.pipe.vae.encode(im_px).latent_dist.mode()
latents.shape

In [14]:
foundation.scheduler

FlowMatchEulerDiscreteScheduler {
  "_class_name": "FlowMatchEulerDiscreteScheduler",
  "_diffusers_version": "0.36.0.dev0",
  "base_image_seq_len": 256,
  "base_shift": 0.5,
  "invert_sigmas": false,
  "max_image_seq_len": 8192,
  "max_shift": 0.9,
  "num_train_timesteps": 1000,
  "shift": 1.0,
  "shift_terminal": 0.02,
  "stochastic_sampling": false,
  "time_shift_type": "exponential",
  "use_beta_sigmas": false,
  "use_dynamic_shifting": true,
  "use_exponential_sigmas": false,
  "use_karras_sigmas": false
}